# 🐱🐶 Image Classification of Cats and Dogs using CNN
**IBM Internship Project by ADITYA VERMA**

This project uses a Convolutional Neural Network (CNN) to classify images of cats and dogs. The model is trained using TensorFlow and evaluated on a validation set. The notebook also allows image upload and prediction to test the model in real-time.


In [ ]:
import os
import zipfile
import pathlib
import urllib.request

# Step 1: Download the ZIP file
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_path = 'cats_and_dogs_filtered.zip'
urllib.request.urlretrieve(url, zip_path)

# Step 2: Extract it manually
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Set the base directory path
base_dir = pathlib.Path('cats_and_dogs_filtered')
print("✅ Dataset extracted at:", base_dir)
# Check that folders are there
print("Folders in base_dir:", os.listdir(base_dir))
print("Folders in train:", os.listdir(base_dir / 'train'))
print("Folders in validation:", os.listdir(base_dir / 'validation'))


✅ Dataset extracted at: cats_and_dogs_filtered
Folders in base_dir: ['train', 'vectorize.py', 'validation']
Folders in train: ['dogs', 'cats']
Folders in validation: ['dogs', 'cats']


In [ ]:
# Define the folder paths
train_cats_dir = base_dir / 'train' / 'cats'
train_dogs_dir = base_dir / 'train' / 'dogs'
val_cats_dir = base_dir / 'validation' / 'cats'
val_dogs_dir = base_dir / 'validation' / 'dogs'

# Count images
print("🐱 Training cats:", len(list(train_cats_dir.glob('*.jpg'))))
print("🐶 Training dogs:", len(list(train_dogs_dir.glob('*.jpg'))))
print("🐱 Validation cats:", len(list(val_cats_dir.glob('*.jpg'))))
print("🐶 Validation dogs:", len(list(val_dogs_dir.glob('*.jpg'))))


🐱 Training cats: 1000
🐶 Training dogs: 1000
🐱 Validation cats: 500
🐶 Validation dogs: 500


In [ ]:
import tensorflow as tf

# Set parameters
batch_size = 32         # Number of images in one batch
img_height = 128        # Resize images to 128 pixels tall
img_width = 128         # Resize images to 128 pixels wide

# Load training data from the folder, resizing images
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir / 'train',          # Path to training folder
    image_size=(img_height, img_width),  # Resize images here
    batch_size=batch_size,
    label_mode='binary'          # Labels as 0 (cat) or 1 (dog)
)

# Load validation data similarly
val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir / 'validation',     # Path to validation folder
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary'
)


Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [ ]:
# Normalize pixel values from [0, 255] to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),   # <-- Add this line first
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

 Trainable params: 12,939,329 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 10  # You can increase later if you want better accuracy

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.5035 - loss: 0.9904 - val_accuracy: 0.5460 - val_loss: 0.6859
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.5444 - loss: 0.6839 - val_accuracy: 0.6170 - val_loss: 0.6463
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.6125 - loss: 0.6483 - val_accuracy: 0.6630 - val_loss: 0.6258
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.6919 - loss: 0.5844 - val_accuracy: 0.6800 - val_loss: 0.6077
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.7289 - loss: 0.5516 - val_accuracy: 0.6780 - val_loss: 0.5990
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.7584 - loss: 0.5031 - val_accuracy: 0.6950 - val_loss: 0.5950
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.7936 - loss: 0.4417 - val_accuracy: 0.6620 - val_loss: 0.7080
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.7802 - loss: 0.4246 - val_accuracy: 0.6870 - val_lo

In [ ]:
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 283ms/step - accuracy: 0.6857 - loss: 0.8665
Validation Loss: 0.8137
Validation Accuracy: 0.7000


In [ ]:
model.save('cats_dogs_model.keras')



In [ ]:
from google.colab import files
files.download('cats_dogs_model.keras')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>